<a href="https://colab.research.google.com/github/srujan7m/summer_analytics/blob/main/Dynamic_Pricing_Complete_Final.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install -U pathway bokeh

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.4/60.4 kB 3.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 149.4/149.4 kB 11.0 MB/s eta 0:00:00
INFO: pip is looking at multiple versions of boto3 to determine which version is compatible with other requirements. This could take a while.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 69.7/69.7 MB 10.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.6/77.6 kB 5.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 777.6/777.6 kB 26.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 139.2/139.2 kB 10.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 26.5/26.5 MB 47.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.5/45.5 kB 3.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 135.3/135.3 kB 10.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 244.6/244.6 kB 18.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 318.4/318.4 kB 

In [ ]:
from google.colab import files

# This opens file picker to upload your CSV dataset
uploaded = files.upload()

Saving dataset.csv to dataset.csv


In [ ]:
import pandas as pd

# Automatically get uploaded filename
filename = list(uploaded.keys())[0]
df_stream = pd.read_csv(filename)
df_stream.head()

,ID,SystemCodeNumber,Capacity,Latitude,Longitude,Occupancy,VehicleType,TrafficConditionNearby,QueueLength,IsSpecialDay,LastUpdatedDate,LastUpdatedTime
0,0,BHMBCCMKT01,577,26.144536,91.736172,61,car,low,1,0,04-10-2016,07:59:00
1,1,BHMBCCMKT01,577,26.144536,91.736172,64,car,low,1,0,04-10-2016,08:25:00
2,2,BHMBCCMKT01,577,26.144536,91.736172,80,car,low,2,0,04-10-2016,08:59:00
3,3,BHMBCCMKT01,577,26.144536,91.736172,107,car,low,2,0,04-10-2016,09:32:00
4,4,BHMBCCMKT01,577,26.144536,91.736172,150,bike,low,2,0,04-10-2016,09:59:00


In [ ]:
import pathway as pw
import numpy as np
from math import radians, cos, sin, asin, sqrt

# Constants
alpha, beta, gamma, delta, epsilon = 1.2, 0.8, 0.5, 1.0, 1.0
lambda_factor = 0.8
base_price = 10
vehicle_weights = {'car': 1.0, 'bike': 0.5, 'truck': 1.5}

# Mock coordinates for 14 lots
lot_locations = {
    f"Lot {i}": (12.95 + i * 0.001, 77.6 + i * 0.001) for i in range(1, 15)
}

def haversine(lat1, lon1, lat2, lon2):
    lon1, lat1, lon2, lat2 = map(radians, [lon1, lat1, lon2, lat2])
    dlon = lon2 - lon1
    dlat = lat2 - lat1
    a = sin(dlat/2)**2 + cos(lat1)*cos(lat2)*sin(dlon/2)**2
    c = 2 * asin(sqrt(a))
    r = 6371
    return c * r

In [ ]:
class ParkingInput(pw.Schema):
    LotID: str
    Occupancy: int
    Capacity: int
    Queue: int
    Traffic: float
    SpecialDay: int
    VehicleType: str
    Timestamp: int

@pw.udf
def compute_price_with_competition(lot_id, occupancy, capacity, queue, traffic, special_day, vehicle_type):
    weight = vehicle_weights.get(vehicle_type, 1.0)
    demand = alpha * (occupancy / capacity) + beta * queue - gamma * traffic + delta * special_day + epsilon * weight
    norm_demand = max(0, min(1, (demand - 1.5) / (10 - 1.5)))
    base = base_price * (1 + lambda_factor * norm_demand)

    lat1, lon1 = lot_locations.get(lot_id, (0, 0))
    nearby_prices = []

    for other_lot, (lat2, lon2) in lot_locations.items():
        if other_lot == lot_id:
            continue
        dist = haversine(lat1, lon1, lat2, lon2)
        if dist <= 1.0:
            simulated_price = base_price * (1 + 0.05 * np.random.randint(0, 5))
            nearby_prices.append(simulated_price)

    avg_competitor = np.mean(nearby_prices) if nearby_prices else base_price

    if occupancy >= capacity and base > avg_competitor:
        base = avg_competitor - 1

    return round(max(5, min(base, 20)), 2)

In [ ]:
t = pw.io.csv.read(filename, schema=ParkingInput, mode="streaming")

out = t.select(
    LotID=t.LotID,
    Price=compute_price_with_competition(
        t.LotID, t.Occupancy, t.Capacity, t.Queue, t.Traffic, t.SpecialDay, t.VehicleType
    ),
    Timestamp=t.Timestamp
)

pw.io.jsonlines.write(out, "out.jsonl")
pw.run()

Output()

/usr/local/lib/python3.11/dist-packages/beartype/_util/hint/pep/utilpeptest.py:311: BeartypeDecorHintPep585DeprecationWarning: PEP 484 type hint typing.Iterable[pathway.internals.expression.ColumnReference] deprecated by PEP 585. This hint is scheduled for removal in the first Python version released after October 5th, 2025. To resolve this, import this hint from "beartype.typing" rather than "typing". For further commentary and alternatives, see also:
    https://beartype.readthedocs.io/en/latest/api_roar/#pep-585-deprecations
  warn(
ERROR:pathway_engine.connectors:Parse error: some fields weren't found in the header (fields present in table: ["ID", "SystemCodeNumber", "Capacity", "Latitude", "Longitude", "Occupancy", "VehicleType", "TrafficConditionNearby", "QueueLength", "IsSpecialDay", "LastUpdatedDate", "LastUpdatedTime"], fields specified in connector: ["LotID", "Occupancy", "Capacity", "Queue", "Traffic", "SpecialDay", "VehicleType", "Timestamp"])
ERROR:pathway_engine.connector

KeyboardInterrupt: 

In [ ]:
import pandas as pd
import json
from bokeh.plotting import figure, output_notebook, show
from bokeh.models import ColumnDataSource

output_notebook()

# Read and load JSON lines
records = []
with open("out.jsonl", "r") as f:
    for line in f:
        records.append(json.loads(line))

# Load into DataFrame
df_out = pd.DataFrame(records)
print("Available columns:", df_out.columns)  # Debug

# Sort by timestamp
df_out = df_out.sort_values(by="Timestamp")

# Filter for one parking lot
selected_lot = "Lot 1"
lot_data = df_out[df_out["LotID"] == selected_lot]

# Plot
source = ColumnDataSource(lot_data)
p = figure(title=f"Dynamic Pricing - {selected_lot}",
           x_axis_label="Timestamp", y_axis_label="Price ($)",
           plot_height=350, plot_width=700)

p.line(x='Timestamp', y='Price', source=source, line_width=2, color='navy')
p.circle(x='Timestamp', y='Price', source=source, size=6, color='red')

show(p)


Available columns: RangeIndex(start=0, stop=0, step=1)


KeyError: 'Timestamp'